# Queue Script

This notebook shows how to validate a script configuration and send it to the script queue using a notebooks.
This is useful when trying to execute an operation that requires a series of scripts to be queued in succession.

Here we will use the high-level class provided by the ts_observatory_control package, which takes care of the majority of the heavy-lifting.

We will use the ATQueue (ScriptQueue with index=2) as an example but this can easily be modified for the MTQueue (ScriptQueue with index=1).

## Importing required libraries

In the cell below, note we import `ATQueue`, and the code to import `MTQueue` is intentionally commented out.
In both cases, the module is imported as `Queue` so the code in the cells below remains functional regardless of the chosen queue.

In [ ]:
from lsst.ts import salobj
import yaml

# For MTQueue use
# from lsst.ts.observatory.control.maintel.mtqueue import MTQueue as Queue
# For ATQueue use
from lsst.ts.observatory.control.auxtel.atqueue import ATQueue as Queue

## Setting up domain and queue

The cell below are simply instantiating a domain to be used by the middleware communication package and a queue object, used for the operations.

In [ ]:
domain = salobj.Domain()

In [ ]:
queue = Queue(domain=domain)

In [ ]:
await queue.start_task

## Basic checks

The next two cells execute liveliness and state checks to ensure the script can be validated and executed successfully.


In [ ]:
await queue.assert_liveliness()

In [ ]:
await queue.assert_all_enabled()

## List available scripts

In [ ]:
await queue.list_external_scripts()

In [ ]:
await queue.list_standard_scripts()

## Defining a configuration

For this exercise, let us pick the standard script `auxtel/track_target_and_take_image.py`.

In [ ]:
is_standard = True
script = "auxtel/track_target_and_take_image.py"

### Get script schema

If you are not familiar with the script you may want to look at the script schema before writting the configuration.

In [ ]:
print(
    await queue.get_script_schema(
        is_standard=is_standard, script=script
    )
)

### Define configuration

Let us now define a configuration and validate it.
We will start with an intentionally invalid configuration that is missing a couple required fields.

Note we define the configuration as a dictionary.

In [ ]:
script_config_bad = dict(
    dec=-85.0,
    ra=0.0,
)

When validating the configuration we will get a `RuntimeError` with a simple description of the problem.
Note that it only outputs one problem at a time so you may have to validate the configuration a couple times until you get it right.

In [ ]:
await queue.validate_config(
    is_standard=is_standard,
    script=script,
    config=script_config_bad,
)

Now let's define a complete configuration,

In [ ]:
script_config = dict(
    dec=-85.0,
    ra=0.0,
    name="Test Target",
    grating="empty_1",
    band_filter="empty_1",
    rot_sky=0.0,
    obs_time=1.0,
    num_exp=1,
    exp_times=[1.0],
)

and validate it.

In [ ]:
await queue.validate_config(
    is_standard=is_standard,
    script=script,
    config=script_config,
)

Although not the primary use-case of this example, at this point, subsequently run via the scriptQueue using the LOVE interface. <br>
The code below converts the config to yaml, so it can then be copy/pasted to the configuration in the LOVE GUI. 

In [ ]:
print(yaml.safe_dump(script_config))

## Queueing Script

Now that we have a valid configuration we can queue the script.

The command below will return as soon as the script is queued, it will not wait for the script to execute.

In [ ]:
await queue.add(
    is_standard=is_standard,
    script=script,
    config=script_config,
)

## Additional queueing options

The Queue class has two additional utility methods to add scripts to the scritpt queue; `add_standard` and `add_external`.

In the example above, you could also have done:

In [ ]:
await queue.add_standard(
    script=script,
    config=script_config,
)

Furthermore, both `add`, `add_standard` and `add_external` have additional options to control the script log level (which is DEBUG by default) and an optional "description" users can add to the script execution metadata.

You can check these plus additional options by adding a `?` to the cell to get the method docstring.

In [ ]:
queue.add?